# Excel Data Ingestion with LakeLogic 

[!["Open In Colab"](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lakelogic/LakeLogic/blob/main/examples/03_data_sources/files/excel/excel_example.ipynb)
[![GitHub Repo](https://img.shields.io/badge/GitHub-Repo-blue?logo=github)](https://github.com/lakelogic/LakeLogic/blob/main/examples/03_data_sources/files/excel/excel_example.ipynb)

## Business Scenario
Your finance team sends you monthly employee data reports in Excel format. These spreadsheets contain:
- Employee contact information
- Salary data
- Department assignments
- Employment status

**The Problem**: Manual Excel uploads often contain data quality issues that break downstream reports.

**The Solution**: Use LakeLogic to automatically validate Excel files before they reach your data warehouse.

## Value Proposition
- **Zero Manual Parsing**: Direct `.xlsx` and `.xls` file reading
- **Instant Validation**: Catch errors before they impact analytics
- **User-Friendly Error Reports**: Send clear feedback to data providers
- **Automated Pipeline**: Transform manual uploads into governed data flows

## 1. Setup
Install LakeLogic with Excel support and set up your environment.

In [ ]:
from pathlib import Path
from lakelogic import DataProcessor

BASE = Path.cwd()
contract_path = BASE / "excel_contract.yaml"
if not contract_path.exists():
    candidate = BASE / "examples" / "03_data_sources" / "files" / "excel" / "excel_contract.yaml"
    if candidate.exists():
        BASE = candidate.parent
        contract_path = candidate

excel_path = BASE / "data" / "employees.xlsx"
csv_fallback = BASE / "data" / "employees.csv"
result = None

def preview_frame(df, title=None, limit=5):
    if title:
        print(title)
    if df is None:
        print("<empty>")
        return
    if hasattr(df, "head"):
        try:
            print(df.head(limit))
            return
        except Exception:
            pass
    if hasattr(df, "limit"):
        try:
            sample = df.limit(limit)
            print(sample)
            return
        except Exception:
            pass
    print(df)


## 2. Preview Excel File Metadata
Let's check what sheets and data the Excel file contains.

In [ ]:
if not excel_path.exists():
    print(f"Missing Excel file: {excel_path.resolve()}")
    if csv_fallback.exists():
        print(f"Sample CSV is available at: {csv_fallback.resolve()}")
        print("Run data/create_excel.py to generate the Excel file, then re-run this cell.")
    else:
        print("Sample data file not found. Run data/create_excel.py to generate it.")
else:
    processor = DataProcessor(contract=contract_path)
    try:
        result = processor.run_source(excel_path)
    except Exception as exc:
        print(f"Excel ingestion failed: {exc}")
        result = None


## 3. Run LakeLogic Validation
Now let's use LakeLogic to validate the Excel data against our contract.

In [ ]:
if result is None:
    print("No results to summarize.")
else:
    print(result)
    print(f"Raw: {len(result.raw)} | Good: {len(result.good)} | Bad: {len(result.bad)}")


## 4. Access Results Using Named Attributes
LakeLogic returns a `ValidationResult` object with clear, named attributes.

In [ ]:
if result is None:
    print("No validated data to display.")
else:
    preview_frame(result.good, "VALIDATED EMPLOYEES (sample):")
    preview_frame(result.bad, "QUARANTINED EMPLOYEES (sample):")


## 5. Inspect Validated Data
Let's see the clean data that passed all validation rules.

## 6. Review Data Quality Issues
For quarantined records, let's see exactly what went wrong.

## 7. Generate User-Friendly Error Report
Create a simple report to send back to your data providers.

## 8. Calculate Data Quality Metrics
Track your data quality over time with key metrics.

## 9. Multi-Sheet Excel Files (Advanced)
Working with Excel files that have multiple sheets.

## Summary

In this tutorial, we demonstrated:

1.  **Native Excel Reading**: Direct `.xlsx` file ingestion without manual conversion
2.  **Named Attributes**: Clean API with `result.good`, `result.bad`, `result.raw`
3.  **Validation Rules**: Email formats, salary ranges, and status checks
4.  **Error Reporting**: Generate user-friendly reports for data providers
5.  **Quality Metrics**: Track data quality with actionable scorecards

### Production Use Cases

**Finance Team Uploads**
```python
result = processor.run_source("monthly_budget.xlsx")
if len(result.bad) > 0:
    send_email_to_finance(result.bad)
else:
    warehouse.write_table("finance.budget", result.good)
```

**HR Data Integration**
```python
result = processor.run_source("employee_updates.xlsx")
metrics = {"pass_rate": len(result.good) / len(result.raw) * 100}
dashboard.update_quality_metrics(metrics)
```

### Next Steps

- **Customize Validation**: Edit `excel_contract.yaml` to match your business rules
- **Automate**: Build a scheduled job to process Excel uploads from a shared folder
- **Materialize**: Use LakeLogic's `materialize` option to save validated data
- **Monitor**: Track data quality trends over time